# Movie Recomendation - Similarity Model

The purpose of this motel is to get recomendations of movies based on a selected movie and related features.

In [1]:
import pandas as pd
import numpy as np

## Importing Database

In [2]:
data = pd.read_csv('../raw_data/treated-db/data.csv')

In [3]:
data.dropna(inplace=True)

In [4]:
data

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,vote_count,vote_average,runtime,release
0,1.0,862,Toy Story (1995),['toy story'],"['adventure', 'animation', 'children', 'comedy...","['tomhanks', 'timallen', 'donrickles']",['johnlasseter'],"led by woody, andy's toys live happily in his ...","['martialarts', 'jealousy', 'toy', 'friendship...",70.910,13173.0,7.9,81.0,1995-10-30
1,2.0,8844,Jumanji (1995),['jumanji'],"['adventure', 'children', 'fantasy']","['robinwilliams', 'jonathanhyde', 'kirstendunst']",['joejohnston'],when siblings judy and peter discover an encha...,"['giantinsect', 'boardgame', 'jungle', 'disapp...",11.919,7711.0,7.2,104.0,1995-12-15
2,3.0,15602,Grumpier Old Men (1995),['grumpier old men'],"['comedy', 'romance']","['waltermatthau', 'jacklemmon', 'ann-margret']",['howarddeutch'],a family wedding reignites the ancient feud be...,"['fishing', 'oldman', 'bestfriend', 'duringcre...",13.122,217.0,6.6,101.0,1995-12-22
3,4.0,31357,Waiting to Exhale (1995),['waiting to exhale'],"['comedy', 'drama', 'romance']","['whitneyhouston', 'angelabassett', 'lorettade...",['forestwhitaker'],"cheated on, mistreated and stepped on, the wom...","['basedonnovelorbook', 'interracialrelationshi...",5.123,79.0,6.2,127.0,1995-12-22
4,5.0,11862,Father of the Bride Part II (1995),['father of the bride part ii'],['comedy'],"['stevemartin', 'dianekeaton', 'martinshort']",['charlesshyer'],just when george banks has recovered from his ...,"['parentchildrelationship', 'baby', 'midlifecr...",15.142,446.0,6.2,106.0,1995-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23308,208843.0,553882,An Acceptable Loss (2019),['an acceptable loss'],"['drama', 'thriller']","['tikasumpter', 'jamieleecurtis', 'bentavassoli']",['joechappelle'],"former top u.s. security adviser elizabeth ""li...","['regret', 'politician', 'betrayal', 'threat',...",7.757,42.0,6.3,102.0,2018-10-13
23309,208863.0,31006,Homicide: The Movie (2000),['homicide: the movie'],"['crime', 'drama']","['danielbaldwin', 'nedbeatty', 'richardbelzer']",['jeandesegonzac'],retired police lieutenant al giardello is runn...,"['usa', 'baltimore']",3.374,12.0,6.9,89.0,2000-02-13
23310,208915.0,639832,Undercover Brother 2 (2019),['undercover brother 2'],"['action', 'comedy']","['michaeljaiwhite', 'barrybostwick', 'jeffdani...",['lesliesmall'],"sixteen years ago, undercover brother and his ...",['spy'],19.101,14.0,3.9,85.0,2019-11-05
23311,208917.0,186021,Caicedo (with Pole) (1894),['caicedo (with pole)'],['documentary'],['juana.caicedo'],['williamk.l.dickson'],king of the slack wire. his daring feats of ba...,"['silentfilm', 'vaudeville', 'acrobats']",1.591,21.0,5.8,1.0,1894-07-25


## Database setup

In [5]:
import ast
def api_to_dict(x):
    return ast.literal_eval(x)

In [6]:
df = data.copy()

In [7]:
df.head(1)

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,vote_count,vote_average,runtime,release
0,1.0,862,Toy Story (1995),['toy story'],"['adventure', 'animation', 'children', 'comedy...","['tomhanks', 'timallen', 'donrickles']",['johnlasseter'],"led by woody, andy's toys live happily in his ...","['martialarts', 'jealousy', 'toy', 'friendship...",70.91,13173.0,7.9,81.0,1995-10-30


In [8]:
import ast
df = data.copy()
# df.drop(columns=['popularity','movieId','vote_count','vote_average','runtime','release'],inplace = True)

df['actors'] = df['actors'].apply(api_to_dict)
df['keywords'] = df['keywords'].apply(api_to_dict)
df['genres'] = df['genres'].apply(api_to_dict)

df.head(1)

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,vote_count,vote_average,runtime,release
0,1.0,862,Toy Story (1995),['toy story'],"[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles]",['johnlasseter'],"led by woody, andy's toys live happily in his ...","[martialarts, jealousy, toy, friendship, bully...",70.91,13173.0,7.9,81.0,1995-10-30


### Overview setup

#### Remove ponctuation and stopwords

In [9]:
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [10]:
def remove_punctuation(x):
    for i in string.punctuation:
        x = x.replace(i, '')
    return x

In [11]:
def stop_words(x):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(x)
    x = [w for w in word_tokens if not w in stop_words] 
  
    return x

In [12]:
df['title_clean'] = df['title_clean'].apply(remove_punctuation)
df['title_clean'] = df['title_clean'].apply(stop_words)
df['overview'] = df['overview'].apply(remove_punctuation)
df['overview'] = df['overview'].apply(stop_words)
df['directors'] = df['directors'].apply(remove_punctuation)
df['directors'] = df['directors'].apply(stop_words)

In [13]:
def to_string(x):
    x = ' '.join(x)
    return x

In [14]:
#Tudão Bolado
df['soup_1'] = (df['genres'] + df['actors'] + df['keywords'] + df['overview'] + df['directors'] + df['title_clean']).map(to_string)

#Sem title
df['soup_2'] = (df['genres'] + df['actors'] + df['keywords'] + df['overview'] + df['directors']).map(to_string)

#Sem overview e title
df['soup_3'] = (df['genres'] + df['actors'] + df['keywords'] + df['directors']).map(to_string)

#Sem Key words
df['soup_4'] = (df['genres'] + df['actors'] + df['overview'] + df['directors'] + df['title_clean']).map(to_string)

#Sem Key words e Title
df['soup_5'] = (df['genres'] + df['actors'] + df['overview'] + df['directors']).map(to_string)

#Sem Actors e Directors e titulo
df['soup_6'] = (df['genres'] + df['keywords'] + df['overview']).map(to_string)

#Genres e KW
df['soup_7'] = (df['genres'] + df['keywords']).map(to_string)

#Genres e Overview
df['soup_8'] = (df['genres'] + df['overview']).map(to_string)

In [15]:
df.head(2)

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,...,runtime,release,soup_1,soup_2,soup_3,soup_4,soup_5,soup_6,soup_7,soup_8
0,1.0,862,Toy Story (1995),"[toy, story]","[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles]",[johnlasseter],"[led, woody, andys, toys, live, happily, room,...","[martialarts, jealousy, toy, friendship, bully...",70.910,...,81.0,1995-10-30,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy ma...,adventure animation children comedy fantasy ma...,adventure animation children comedy fantasy le...
1,2.0,8844,Jumanji (1995),[jumanji],"[adventure, children, fantasy]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[siblings, judy, peter, discover, enchanted, b...","[giantinsect, boardgame, jungle, disappearance...",11.919,...,104.0,1995-12-15,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy giantinsect boardga...,adventure children fantasy giantinsect boardga...,adventure children fantasy siblings judy peter...


In [16]:
df.iloc[0].soup_1

'adventure animation children comedy fantasy tomhanks timallen donrickles martialarts jealousy toy friendship bullying elementaryschool friends rivalry rescue mission walkietalkie boynextdoor newtoy neighborhood toycomestolife resourcefulness led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences johnlasseter toy story'

In [17]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count1 = CountVectorizer()
count_matrix1 = count1.fit_transform(df['soup_1'])

count2 = CountVectorizer()
count_matrix2 = count2.fit_transform(df['soup_2'])

count3 = CountVectorizer()
count_matrix3 = count3.fit_transform(df['soup_3'])

count4 = CountVectorizer()
count_matrix4 = count4.fit_transform(df['soup_4'])

count5 = CountVectorizer()
count_matrix5 = count5.fit_transform(df['soup_5'])

count6 = CountVectorizer()
count_matrix6 = count6.fit_transform(df['soup_6'])

count7 = CountVectorizer()
count_matrix7 = count7.fit_transform(df['soup_7'])

count8 = CountVectorizer()
count_matrix8 = count8.fit_transform(df['soup_8'])

In [18]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim1 = cosine_similarity(count_matrix1, count_matrix1)
cosine_sim2 = cosine_similarity(count_matrix2, count_matrix2)
cosine_sim3 = cosine_similarity(count_matrix3, count_matrix3)
cosine_sim4 = cosine_similarity(count_matrix4, count_matrix4)
cosine_sim5 = cosine_similarity(count_matrix5, count_matrix5)
cosine_sim6 = cosine_similarity(count_matrix6, count_matrix6)
cosine_sim7 = cosine_similarity(count_matrix7, count_matrix7)
cosine_sim8 = cosine_similarity(count_matrix8, count_matrix8)

In [19]:
cosine_sim1.shape

(23308, 23308)

In [20]:
df2 = df.copy()

In [21]:
# Reset index of our main DataFrame and construct reverse mapping as before
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [22]:
indices

title
Toy Story (1995)                          0
Jumanji (1995)                            1
Grumpier Old Men (1995)                   2
Waiting to Exhale (1995)                  3
Father of the Bride Part II (1995)        4
                                      ...  
An Acceptable Loss (2019)             23303
Homicide: The Movie (2000)            23304
Undercover Brother 2 (2019)           23305
Caicedo (with Pole) (1894)            23306
Klaus (2019)                          23307
Length: 23308, dtype: int64

In [23]:
sim_scores = list(enumerate(cosine_sim1[0]))
sim_scores

[(0, 0.9999999999999991),
 (1, 0.05773502691896257),
 (2, 0.015357377920848781),
 (3, 0.03450327796711771),
 (4, 0.016854996561581053),
 (5, 0.0),
 (6, 0.02076136996343499),
 (7, 0.052704627669472995),
 (8, 0.017677669529663688),
 (9, 0.013085598064755342),
 (10, 0.016308201826336056),
 (11, 0.018633899812498245),
 (12, 0.060241449667687415),
 (13, 0.0),
 (14, 0.014680505487867585),
 (15, 0.013085598064755342),
 (16, 0.0),
 (17, 0.05175491695067656),
 (18, 0.015504341823651057),
 (19, 0.018633899812498245),
 (20, 0.03405574568898749),
 (21, 0.0),
 (22, 0.024999999999999998),
 (23, 0.0),
 (24, 0.01507556722888818),
 (25, 0.044721359549995794),
 (26, 0.016666666666666666),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.044721359549995794),
 (31, 0.04008918628686366),
 (32, 0.01419904585617662),
 (33, 0.03627381250550058),
 (34, 0.014085904245475274),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.01784576525620624),
 (39, 0.03773659540699707),
 (40, 0.015357377920848781),
 (41, 0.0121267812518

In [124]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim):
    
    # idx Get the index of the movie that matches the title
    idx = indices[title]

    # Sim_scores creates a list of all movies and the cosine similarity related to the movie selected in 'title'
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on the similarity scores
    # Note: Key is the field that the sort fuction will use to do the sort (position 1 of the tuple)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the X most similar movies (position 0 will be the selected movie)
    sim_scores = sim_scores[1:30]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Get the movie cosine scores:
    cosine_scores = [i[1] for i in sim_scores]
    
    # Return the top 10 most similar movies
    #return df2['title'].iloc[movie_indices]
    a=pd.DataFrame(df2[['genres','title','actors','directors','vote_average','popularity']].iloc[movie_indices]).reset_index(drop=True)
    b=pd.DataFrame(cosine_scores, columns=['cosine_score'])
    
    return pd.concat([a,b], axis = 1).set_index('title')

In [51]:
pd.DataFrame(df2[['title','vote_average','popularity']].iloc[0]).transpose()

,title,vote_average,popularity
0,Toy Story (1995),7.9,70.91


In [35]:
df2.keys()

Index(['index', 'movieId', 'tmdbId', 'title', 'title_clean', 'genres',
       'actors', 'directors', 'overview', 'keywords', 'popularity',
       'vote_count', 'vote_average', 'runtime', 'release', 'soup_1', 'soup_2',
       'soup_3', 'soup_4', 'soup_5', 'soup_6', 'soup_7', 'soup_8'],
      dtype='object')

In [149]:
print('Everything stronger')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim1)

Everything stronger


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.355161
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.308899
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.241308
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.237835
Batman and Robin (1949),"[action, adventure, crime, drama, sci-fi]","[robertlowery, johnnyduncan, lyletalbot]",[spencergordonbennet],5.8,3.864,0.237096
Batman Forever (1995),"[action, adventure, comedy, crime]","[valkilmer, tommyleejones, jimcarrey]",[joelschumacher],5.4,21.650,0.223353
DC Showcase: Catwoman (2011),"[action, adventure, animation, sci-fi]","[elizadushku, johndimaggio, lilianamumy]",[laurenmontgomery],6.9,9.221,0.217113
Death Wish (1974),"[action, crime, drama]","[charlesbronson, hopelange, vincentgardenia]",[michaelwinner],7.0,13.590,0.211950
Batman (1989),"[action, crime, thriller]","[jacknicholson, michaelkeaton, kimbasinger]",[timburton],7.2,43.047,0.210310


In [148]:
print('Everything stronger')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim1)

Everything stronger


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.276648
American Hustle (2013),"[crime, drama]","[christianbale, bradleycooper, jeremyrenner]",[davidorussell],6.8,17.433,0.195006
Confessions of a Nazi Spy (1939),[drama],"[edwardg.robinson, francislederer, georgesanders]",[anatolelitvak],6.2,2.566,0.188639
"X-Files: I Want to Believe, The (2008)","[drama, mystery, sci-fi, thriller]","[davidduchovny, gilliananderson, amandapeet]",[chriscarter],5.6,14.209,0.184789
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.182018
Frank and Cindy (2015),"[comedy, drama]","[renerusso, oliverplatt, johnnysimmons]",[gjechternkamp],5.5,4.900,0.175035
Black Mass (2015),"[crime, drama]","[johnnydepp, joeledgerton, juliannenicholson]",[scottcooper],6.4,11.496,0.168408
Manhunter (1986),"[action, crime, drama, horror, thriller]","[williampetersen, kimgreist, dennisfarina]",[michaelmann],7.0,0.600,0.165168
Mississippi Burning (1988),"[crime, drama, thriller]","[genehackman, willemdafoe, francesmcdormand]",[alanparker],7.7,10.558,0.165025


In [129]:
print('Without title')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim2)

Without title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.326377
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.303615
Batman and Robin (1949),"[action, adventure, crime, drama, sci-fi]","[robertlowery, johnnyduncan, lyletalbot]",[spencergordonbennet],5.8,3.864,0.243267
DC Showcase: Catwoman (2011),"[action, adventure, animation, sci-fi]","[elizadushku, johndimaggio, lilianamumy]",[laurenmontgomery],6.9,9.221,0.229095
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.221163
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.220707
LEGO DC Comics Super Heroes: Batman: Be-Leaguered (2014),"[action, adventure, animation, children]","[troybaker, deebradleybaker, johndimaggio]",[rickmorales],6.3,7.566,0.216730
Teenage Mutant Ninja Turtles (2014),"[action, adventure, comedy]","[meganfox, willarnett, williamfichtner]",[jonathanliebesman],5.9,76.397,0.212486
LEGO DC: Batman - Family Matters (2019),"[animation, children]","[troybaker, jasonspisak, willfriedle]",[mattpeters],7.1,35.273,0.209134


In [150]:
print('Without title')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim2)

Without title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.226274
Confessions of a Nazi Spy (1939),[drama],"[edwardg.robinson, francislederer, georgesanders]",[anatolelitvak],6.2,2.566,0.211058
American Hustle (2013),"[crime, drama]","[christianbale, bradleycooper, jeremyrenner]",[davidorussell],6.8,17.433,0.203200
"X-Files: I Want to Believe, The (2008)","[drama, mystery, sci-fi, thriller]","[davidduchovny, gilliananderson, amandapeet]",[chriscarter],5.6,14.209,0.191237
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.187083
Black Mass (2015),"[crime, drama]","[johnnydepp, joeledgerton, juliannenicholson]",[scottcooper],6.4,11.496,0.173205
Punisher: War Zone (2008),"[action, crime, drama, thriller]","[raystevenson, dominicwest, juliebenz]",[lexialexander],5.7,18.373,0.171791
10 Minutes Gone (2019),"[action, crime, mystery, thriller]","[brucewillis, michaelchiklis, meadowwilliams]",[brianamiller],4.6,13.632,0.171499
White Boy Rick (2018),"[crime, drama]","[richiemerritt, matthewmcconaughey, jenniferja...",[yanndemange],6.3,15.913,0.169775


In [130]:
print('Everything except overview and title')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim3)

Everything except overview and title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.503953
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.466667
Superman Returns (2006),"[action, adventure, sci-fi, imax]","[brandonrouth, kevinspacey, katebosworth]",[bryansinger],5.6,24.329,0.267652
Man of Steel (2013),"[action, adventure, fantasy, sci-fi, imax]","[henrycavill, amyadams, michaelshannon]",[zacksnyder],6.5,55.745,0.267652
Kick-Ass 2 (2013),"[action, comedy, crime]","[aarontaylor-johnson, chloëgracemoretz, christ...",[jeffwadlow],6.4,29.675,0.258199
Batman (1943),"[action, adventure, crime, sci-fi, thriller]","[lewiswilson, douglascroft, j.carrolnaish]",[lamberthillyer],6.3,3.574,0.250000
Thor: The Dark World (2013),"[action, adventure, fantasy, imax]","[chrishemsworth, natalieportman, tomhiddleston]",[alantaylor],6.6,46.641,0.250000
Batman v Superman: Dawn of Justice (2016),"[action, adventure, fantasy, sci-fi]","[benaffleck, henrycavill, jesseeisenberg]",[zacksnyder],5.9,70.312,0.250000
Star Wars: Episode VII - The Force Awakens (2015),"[action, adventure, fantasy, sci-fi, imax]","[harrisonford, markhamill, carriefisher]",[jjabrams],7.4,89.419,0.242536


In [151]:
print('Everything except overview and title')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim3)

Everything except overview and title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
J. Edgar (2011),[drama],"[leonardodicaprio, joshhamilton, geoffpierson]",[clinteastwood],6.2,14.972,0.333333
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.316228
"Opportunists, The (2000)","[comedy, crime, drama]","[christopherwalken, petermcdonald, kateburton]",[mylesconnell],5.6,3.061,0.283473
The Most Hated Woman in America (2017),"[crime, drama]","[melissaleo, joshlucas, michaelchernus]",[tommyohaver],6.1,9.411,0.283473
Breathe (2017),[drama],"[andrewgarfield, clairefoy, hughbonneville]",[andyserkis],7.4,15.208,0.283473
Aaliyah: The Princess of R&B (2014),[drama],"[alexandrashipp, clébennett, eliseneal]",[bradleywalsh],5.0,7.070,0.283473
King Cobra (2016),"[crime, drama]","[garrettclayton, christianslater, keeganallen]",[justinkelly],5.8,11.390,0.279508
The Post (2017),"[drama, thriller]","[merylstreep, tomhanks, bobodenkirk]",[stevenspielberg],7.0,15.104,0.279508
Vita & Virginia (2019),"[drama, romance]","[elizabethdebicki, gemmaarterton, isabellaross...",[chanyabutton],6.5,4.844,0.277350


In [131]:
print('Everything except key words')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim4)

Everything except key words


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.337580
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.281695
Batman Forever (1995),"[action, adventure, comedy, crime]","[valkilmer, tommyleejones, jimcarrey]",[joelschumacher],5.4,21.650,0.277735
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.245050
Batman Beyond Darwyn Cooke's Batman 75th Anniversary Short (2014),"[action, animation, sci-fi]","[willfriedle, kevinconroy]","[brucetimm, darwyncooke]",7.5,3.399,0.234148
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.219265
Batman (1989),"[action, crime, thriller]","[jacknicholson, michaelkeaton, kimbasinger]",[timburton],7.2,43.047,0.209923
Batman and Robin (1949),"[action, adventure, crime, drama, sci-fi]","[robertlowery, johnnyduncan, lyletalbot]",[spencergordonbennet],5.8,3.864,0.203810
Batman Returns (1992),"[action, crime]","[michaelkeaton, dannydevito, michellepfeiffer]",[timburton],6.8,27.871,0.198479


In [152]:
print('Everything except key words')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim4)

Everything except key words


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.228802
Frank and Cindy (2015),"[comedy, drama]","[renerusso, oliverplatt, johnnysimmons]",[gjechternkamp],5.5,4.900,0.203363
The Irishman (2019),"[crime, drama]","[robertdeniro, alpacino, joepesci]",[martinscorsese],7.7,46.494,0.190885
10 Minutes Gone (2019),"[action, crime, mystery, thriller]","[brucewillis, michaelchiklis, meadowwilliams]",[brianamiller],4.6,13.632,0.187416
Everybody's Fine (2009),[drama],"[robertdeniro, drewbarrymore, katebeckinsale]",[kirkjones],6.7,10.910,0.181164
Undercover Man (1949),"[crime, drama, film-noir, romance]","[glennford, ninafoch, jameswhitmore]",[josephhlewis],6.3,3.361,0.177394
This Woman Is Dangerous (1952),"[crime, drama]","[joancrawford, dennismorgan, davidbrian]",[felixefeist],6.0,1.741,0.177229
Act of Violence (1948),"[drama, film-noir, thriller]","[vanheflin, robertryan, janetleigh]",[fredzinnemann],6.8,3.567,0.170935
The Diary of Anne Frank (1959),"[drama, war]","[millieperkins, josephschildkraut, shelleywint...",[georgestevens],7.1,11.502,0.170697


In [132]:
print('Everything except key words and title')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim5)

Everything except key words and title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.291343
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.270226
Batman Forever (1995),"[action, adventure, comedy, crime]","[valkilmer, tommyleejones, jimcarrey]",[joelschumacher],5.4,21.650,0.238849
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.214423
LEGO DC Comics Super Heroes: Batman: Be-Leaguered (2014),"[action, adventure, animation, children]","[troybaker, deebradleybaker, johndimaggio]",[rickmorales],6.3,7.566,0.200574
LEGO DC: Batman - Family Matters (2019),"[animation, children]","[troybaker, jasonspisak, willfriedle]",[mattpeters],7.1,35.273,0.199754
Batman (1989),"[action, crime, thriller]","[jacknicholson, michaelkeaton, kimbasinger]",[timburton],7.2,43.047,0.196554
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.195064
DC Showcase: Catwoman (2011),"[action, adventure, animation, sci-fi]","[elizadushku, johndimaggio, lilianamumy]",[laurenmontgomery],6.9,9.221,0.191655


In [153]:
print('Everything except key words and title')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim5)

Everything except key words and title


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
10 Minutes Gone (2019),"[action, crime, mystery, thriller]","[brucewillis, michaelchiklis, meadowwilliams]",[brianamiller],4.6,13.632,0.196722
The Irishman (2019),"[crime, drama]","[robertdeniro, alpacino, joepesci]",[martinscorsese],7.7,46.494,0.195100
Everybody's Fine (2009),[drama],"[robertdeniro, drewbarrymore, katebeckinsale]",[kirkjones],6.7,10.910,0.187317
Undercover Man (1949),"[crime, drama, film-noir, romance]","[glennford, ninafoch, jameswhitmore]",[josephhlewis],6.3,3.361,0.185419
This Woman Is Dangerous (1952),"[crime, drama]","[joancrawford, dennismorgan, davidbrian]",[felixefeist],6.0,1.741,0.184211
Frank and Cindy (2015),"[comedy, drama]","[renerusso, oliverplatt, johnnysimmons]",[gjechternkamp],5.5,4.900,0.183533
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.181369
City by the Sea (2002),"[crime, drama]","[robertdeniro, francesmcdormand, jamesfranco]",[michaelcatonjones],5.8,8.316,0.177343
Act of Violence (1948),"[drama, film-noir, thriller]","[vanheflin, robertryan, janetleigh]",[fredzinnemann],6.8,3.567,0.177343


In [133]:
print('Everything except actors and directors')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim6)

Everything except actors and directors


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.305697
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.276956
Batman and Robin (1949),"[action, adventure, crime, drama, sci-fi]","[robertlowery, johnnyduncan, lyletalbot]",[spencergordonbennet],5.8,3.864,0.263822
DC Showcase: Catwoman (2011),"[action, adventure, animation, sci-fi]","[elizadushku, johndimaggio, lilianamumy]",[laurenmontgomery],6.9,9.221,0.248898
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.244505
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.241747
LEGO DC Comics Super Heroes: Batman: Be-Leaguered (2014),"[action, adventure, animation, children]","[troybaker, deebradleybaker, johndimaggio]",[rickmorales],6.3,7.566,0.239357
LEGO DC: Batman - Family Matters (2019),"[animation, children]","[troybaker, jasonspisak, willfriedle]",[mattpeters],7.1,35.273,0.224807
Teenage Mutant Ninja Turtles (2014),"[action, adventure, comedy]","[meganfox, willarnett, williamfichtner]",[jonathanliebesman],5.9,76.397,0.223057


In [154]:
print('Everything except actors and directors')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim6)

Everything except actors and directors


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.257396
Confessions of a Nazi Spy (1939),[drama],"[edwardg.robinson, francislederer, georgesanders]",[anatolelitvak],6.2,2.566,0.250319
American Hustle (2013),"[crime, drama]","[christianbale, bradleycooper, jeremyrenner]",[davidorussell],6.8,17.433,0.227002
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.210675
"X-Files: I Want to Believe, The (2008)","[drama, mystery, sci-fi, thriller]","[davidduchovny, gilliananderson, amandapeet]",[chriscarter],5.6,14.209,0.202289
Dr. Ehrlich's Magic Bullet (1940),[drama],"[edwardg.robinson, ruthgordon, ottokruger]",[williamdieterle],6.3,3.480,0.197814
White Boy Rick (2018),"[crime, drama]","[richiemerritt, matthewmcconaughey, jenniferja...",[yanndemange],6.3,15.913,0.188434
Black Mass (2015),"[crime, drama]","[johnnydepp, joeledgerton, juliannenicholson]",[scottcooper],6.4,11.496,0.187663
Tortured (2008),"[crime, thriller]","[colehauser, laurencefishburne, jamescromwell]",[nolanlebovitz],5.4,5.861,0.185369


In [155]:
print('Only Genres and Keywords')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim7)

Only Genres and Keywords


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
The Most Hated Woman in America (2017),"[crime, drama]","[melissaleo, joshlucas, michaelchernus]",[tommyohaver],6.1,9.411,0.500000
Breathe (2017),[drama],"[andrewgarfield, clairefoy, hughbonneville]",[andyserkis],7.4,15.208,0.500000
Aaliyah: The Princess of R&B (2014),[drama],"[alexandrashipp, clébennett, eliseneal]",[bradleywalsh],5.0,7.070,0.500000
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.471405
I Want to Live! (1958),"[crime, drama]","[susanhayward, simonoakland, virginiavincent]",[robertwise],7.1,4.589,0.433013
Carbine Williams (1952),"[crime, drama]","[jamesstewart, jeanhagen, wendellcorey]",[richardthorpe],6.1,2.983,0.433013
Madeleine (1950),"[crime, drama]","[anntodd, normanwooland, ivandesny]",[davidlean],6.7,2.286,0.433013
"Client List, The (2010)",[drama],"[jenniferlovehewitt, teddysears, sonjabennett]",[ericlaneuville],5.6,10.175,0.433013
The Riverman (2004),"[crime, drama, thriller]","[brucegreenwood, samjaeger, sarahmanninen]",[billeagles],5.8,4.914,0.433013


In [156]:
print('Only Genres and Keywords')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim7)

Only Genres and Keywords


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
The Most Hated Woman in America (2017),"[crime, drama]","[melissaleo, joshlucas, michaelchernus]",[tommyohaver],6.1,9.411,0.500000
Breathe (2017),[drama],"[andrewgarfield, clairefoy, hughbonneville]",[andyserkis],7.4,15.208,0.500000
Aaliyah: The Princess of R&B (2014),[drama],"[alexandrashipp, clébennett, eliseneal]",[bradleywalsh],5.0,7.070,0.500000
Driven (2019),"[drama, thriller]","[jasonsudeikis, leepace, judygreer]",[nickhamm],6.3,11.000,0.471405
I Want to Live! (1958),"[crime, drama]","[susanhayward, simonoakland, virginiavincent]",[robertwise],7.1,4.589,0.433013
Carbine Williams (1952),"[crime, drama]","[jamesstewart, jeanhagen, wendellcorey]",[richardthorpe],6.1,2.983,0.433013
Madeleine (1950),"[crime, drama]","[anntodd, normanwooland, ivandesny]",[davidlean],6.7,2.286,0.433013
"Client List, The (2010)",[drama],"[jenniferlovehewitt, teddysears, sonjabennett]",[ericlaneuville],5.6,10.175,0.433013
The Riverman (2004),"[crime, drama, thriller]","[brucegreenwood, samjaeger, sarahmanninen]",[billeagles],5.8,4.914,0.433013


In [135]:
print('Only Genres and Overview')
print('=============================================================')
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim8)

Only Genres and Overview


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Dark Knight, The (2008)","[action, crime, drama, imax]","[christianbale, heathledger, michaelcaine]",[christophernolan],8.5,66.543,0.255344
Batman Forever (1995),"[action, adventure, comedy, crime]","[valkilmer, tommyleejones, jimcarrey]",[joelschumacher],5.4,21.650,0.255155
Batman: Gotham by Gaslight (2018),"[action, animation, sci-fi, thriller]","[brucegreenwood, jennifercarpenter, anthonyste...",[samliu],6.6,30.941,0.243432
LEGO DC Comics Super Heroes: Batman: Be-Leaguered (2014),"[action, adventure, animation, children]","[troybaker, deebradleybaker, johndimaggio]",[rickmorales],6.3,7.566,0.231034
Batman Begins (2005),"[action, crime, imax]","[christianbale, liamneeson, michaelcaine]",[christophernolan],7.7,49.706,0.223607
LEGO DC: Batman - Family Matters (2019),"[animation, children]","[troybaker, jasonspisak, willfriedle]",[mattpeters],7.1,35.273,0.219971
DC Showcase: Catwoman (2011),"[action, adventure, animation, sci-fi]","[elizadushku, johndimaggio, lilianamumy]",[laurenmontgomery],6.9,9.221,0.218537
Batman vs. Two-Face (2017),[animation],"[adamwest, burtward, williamshatner]",[rickmorales],6.2,17.090,0.217732
Batman and Robin (1949),"[action, adventure, crime, drama, sci-fi]","[robertlowery, johnnyduncan, lyletalbot]",[spencergordonbennet],5.8,3.864,0.213003


In [157]:
print('Only Genres and Overview')
print('=============================================================')
get_recommendations('Catch Me If You Can (2002)', cosine_sim8)

Only Genres and Overview


,genres,actors,directors,vote_average,popularity,cosine_score
title,,,,,,
"Wings of Eagles, The (1957)","[drama, war]","[johnwayne, dandailey, maureeno'hara]",[johnford],6.4,3.361,0.228665
The Threat (1949),"[crime, thriller]","[michaelo'shea, virginiagrey, charlesmcgraw]",[felixefeist],5.8,3.010,0.216930
"FBI Story, The (1959)","[crime, drama, thriller]","[jamesstewart, veramiles, murrayhamilton]",[mervynleroy],6.2,3.083,0.214373
10 Minutes Gone (2019),"[action, crime, mystery, thriller]","[brucewillis, michaelchiklis, meadowwilliams]",[brianamiller],4.6,13.632,0.214373
The Irishman (2019),"[crime, drama]","[robertdeniro, alpacino, joepesci]",[martinscorsese],7.7,46.494,0.214043
Everybody's Fine (2009),[drama],"[robertdeniro, drewbarrymore, katebeckinsale]",[kirkjones],6.7,10.910,0.206835
Dr. Ehrlich's Magic Bullet (1940),[drama],"[edwardg.robinson, ruthgordon, ottokruger]",[williamdieterle],6.3,3.480,0.206835
This Woman Is Dangerous (1952),"[crime, drama]","[joancrawford, dennismorgan, davidbrian]",[felixefeist],6.0,1.741,0.205882
Undercover Man (1949),"[crime, drama, film-noir, romance]","[glennford, ninafoch, jameswhitmore]",[josephhlewis],6.3,3.361,0.202670
